In [155]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from textblob import TextBlob
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shashu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/shashu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [156]:
df = pd.read_csv("cyberbullying_tweets.csv")

In [157]:
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [158]:
df.duplicated().sum()
df.drop_duplicates(keep='first', inplace=True)

In [159]:
def clean_tweet_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'#', '', text)  # Remove hashtag symbol
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters and numbers
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespaces
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')]) # Remove stopwords
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()]) # Lemmatization
    return text

df['cleaned_tweet_text'] = df['tweet_text'].apply(clean_tweet_text)
df[['tweet_text', 'cleaned_tweet_text']].head()
df.drop("tweet_text", axis=1, inplace=True)

In [160]:
X = df['cleaned_tweet_text']
y = df['cyberbullying_type']

In [161]:
knn = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('svd', TruncatedSVD(n_components=100)),
    ('clf', KNeighborsClassifier(n_neighbors=9, weights='uniform', metric='euclidean'))
])

In [162]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [163]:
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred))

                     precision    recall  f1-score   support

                age       0.94      0.94      0.94      1602
          ethnicity       0.98      0.95      0.96      1636
             gender       0.85      0.77      0.81      1514
  not_cyberbullying       0.48      0.52      0.50      1624
other_cyberbullying       0.50      0.54      0.52      1594
           religion       0.94      0.88      0.91      1562

           accuracy                           0.77      9532
          macro avg       0.78      0.77      0.77      9532
       weighted avg       0.78      0.77      0.77      9532

